In [1]:
import pandas as pd
import googlemaps
import time
from tqdm import tqdm
import os
from dotenv import load_dotenv

In [2]:
# --- CONFIGURATION ---

# This command looks for a .env file in your project's root directory
# and loads the variables from it into the environment.
load_dotenv()

# Now, securely get the API key from the environment variables.
# os.getenv() will return None if the key isn't found.
API_KEY = os.getenv('GOOGLE_API_KEY')

# A robust check to make sure the API key was loaded correctly.
if not API_KEY:
    raise ValueError("Google API Key not found. Make sure you have a .env file in the root directory with GOOGLE_API_KEY='YourKey'")
else:
    print("API Key successfully loaded from .env file.")

API Key successfully loaded from .env file.


In [3]:
# Initialize the Google Maps client
gmaps = googlemaps.Client(key=API_KEY)

# Define file paths
input_path = '../data/processed/nyc_sales_analysis_ready.parquet'
output_path = '../data/processed/nyc_sales_geocoded.parquet'
checkpoint_path = '../data/processed/nyc_sales_geocoded_checkpoint.parquet'

In [4]:
# --- 1. Load the Analysis-Ready Dataset ---
df = pd.read_parquet(input_path)
print(f"Loaded {len(df)} properties to geocode.")

# --- 2. Define the Geocoding Function ---
def geocode_address(address, borough):
    """
    Geocodes a single address using the Google Geocoding API.
    Includes the borough for better accuracy in NYC.
    """
    full_query = f"{address}, {borough}, NY"
    try:
        geocode_result = gmaps.geocode(full_query)
        if geocode_result:
            location = geocode_result[0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            return None, None
    except Exception as e:
        print(f"An error occurred for address '{full_query}': {e}")
        return None, None

# --- 3. Implement Checkpointing to Handle Interruptions ---
# Check if a checkpoint file already exists. If so, load it and only process the remaining rows.
try:
    df_checkpoint = pd.read_parquet(checkpoint_path)
    # Get the addresses we've already processed
    processed_indices = df_checkpoint.index
    # Select only the rows from the original dataframe that have NOT been processed
    df_to_process = df.drop(processed_indices)
    print(f"Loaded checkpoint. {len(df_checkpoint)} addresses already geocoded.")
    print(f"Resuming with {len(df_to_process)} remaining addresses.")
except FileNotFoundError:
    print("No checkpoint file found. Starting from scratch.")
    df_to_process = df.copy()
    df_checkpoint = pd.DataFrame()


# --- 4. RUN ON THE FULL DATASET (WITH CHECKPOINTING) ---
if not df_to_process.empty:
    print(f"\n--- Starting geocoding for {len(df_to_process)} properties. This may take a while... ---")
    tqdm.pandas(desc="Geocoding Progress")

    # This loop will run for the remaining addresses and save progress every 500 rows
    results = []
    for index, row in tqdm(df_to_process.iterrows(), total=len(df_to_process)):
        lat, lng = geocode_address(row['address'], row['borough_name'])
        results.append({'latitude': lat, 'longitude': lng})
        
        # Save progress every 500 iterations
        if (len(results) % 500 == 0):
            temp_df = pd.DataFrame(results, index=df_to_process.index[:len(results)])
            df_to_save = pd.concat([df_checkpoint, temp_df])
            df_to_save.to_parquet(checkpoint_path)
            # print(f"Saved checkpoint with {len(df_to_save)} rows.")
            
    # Combine results for the processed batch
    final_batch_results = pd.DataFrame(results, index=df_to_process.index)
    
    # Combine with any previously checkpointed data
    df_final = pd.concat([df_checkpoint, final_batch_results])
    
    # --- 5. Save the Final Enriched Data ---
    print(f"\nGeocoding complete. Saving {len(df_final)} rows to {output_path}")
    # Join the lat/lon data back to the original dataframe
    df_geocoded = df.join(df_final[['latitude', 'longitude']])
    df_geocoded.to_parquet(output_path)
    print("Final file saved successfully!")

    # Optional: remove the checkpoint file after successful completion
    # os.remove(checkpoint_path)
else:
    print("\nAll addresses have already been geocoded. Loading final file.")
    df_geocoded = pd.read_parquet(output_path)

display(df_geocoded.head())

Loaded 164865 properties to geocode.
No checkpoint file found. Starting from scratch.

--- Starting geocoding for 164865 properties. This may take a while... ---


  0%|                                                                                            | 5/164865 [00:00<2:15:28, 20.28it/s]

An error occurred for address '18 EDWIN STREET, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '10 EDWIN STREET, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '524 SYCAMORE STREET, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '7 WEAVER STREET, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '260 SHIRLEY AVENUE, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '111 LENZIE STREET, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '1634 ARDEN AVENUE, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )


  0%|                                                                                           | 12/164865 [00:00<1:38:25, 27.91it/s]

An error occurred for address '7 SANDGAP STREET, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '50 MAY PLACE, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '22 MAY PLACE, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '60 SEGUINE PLACE, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '212 SNEDEN AVENUE, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '36 LUCY LOOP, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '137 FABIAN ST, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '179 BARCLAY AVENUE, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )


...

  0%|▏                                                                                         | 269/164865 [00:08<1:48:06, 25.37it/s]

An error occurred for address '94 MARYLAND AVENUE, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '39 NEW LANE, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '31 NEW LANE, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '58 SCARBORO AVE, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '83 HARBOR VIEW PLACE S, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '1467 BAY STREET, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '100 WADSWORTH ROAD, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )
An error occurred for address '82 MARYLAND AVENUE, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )


  0%|▏                                                                                         | 272/164865 [00:08<1:26:02, 31.88it/s]


An error occurred for address '78 MARYLAND AVENUE, Staten Island, NY': REQUEST_DENIED (The provided API key is invalid. )


KeyboardInterrupt: 